<a href="https://colab.research.google.com/github/yadavnaman/Ensemble_Learning/blob/master/TOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

In [2]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip"
!unzip 'bank.zip'

--2021-03-26 04:22:49--  https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 579043 (565K) [application/x-httpd-php]
Saving to: ‘bank.zip’

bank.zip            100%[===================>] 565.47K  2.17MB/s    in 0.3s    

2021-03-26 04:22:49 (2.17 MB/s) - ‘bank.zip’ saved [579043/579043]

Archive:  bank.zip
  inflating: bank-full.csv           
  inflating: bank-names.txt          
  inflating: bank.csv                


In [3]:
dataset=pd.read_csv('bank-full.csv',sep=';')
dataset=pd.DataFrame(dataset)



In [4]:

Y=dataset.iloc[:,-1]
X=dataset.iloc[:,:-1]
X=X.drop(['contact'],axis='columns')
X

,age,job,marital,education,default,balance,housing,loan,day,month,duration,campaign,pdays,previous,poutcome
0,58,management,married,tertiary,no,2143,yes,no,5,may,261,1,-1,0,unknown
1,44,technician,single,secondary,no,29,yes,no,5,may,151,1,-1,0,unknown
2,33,entrepreneur,married,secondary,no,2,yes,yes,5,may,76,1,-1,0,unknown
3,47,blue-collar,married,unknown,no,1506,yes,no,5,may,92,1,-1,0,unknown
4,33,unknown,single,unknown,no,1,no,no,5,may,198,1,-1,0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,17,nov,977,3,-1,0,unknown
45207,71,retired,divorced,primary,no,1729,no,no,17,nov,456,2,-1,0,unknown
45208,72,retired,married,secondary,no,5715,no,no,17,nov,1127,5,184,3,success
45209,57,blue-collar,married,secondary,no,668,no,no,17,nov,508,4,-1,0,unknown


In [5]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder()
X=pd.get_dummies(X)

In [6]:
X=pd.DataFrame(X)

In [7]:
X.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,2143,5,261,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,44,29,5,151,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,33,2,5,76,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,47,1506,5,92,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,33,1,5,198,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [8]:
Y=pd.get_dummies(Y);
Y

,no,yes
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
45206,0,1
45207,0,1
45208,0,1
45209,1,0


In [9]:
Y=Y.drop(['no'],axis='columns')
Y


,yes
0,0
1,0
2,0
3,0
4,0
...,...
45206,1
45207,1
45208,1
45209,0


In [10]:
from sklearn.preprocessing import StandardScaler


In [11]:
from sklearn.model_selection import train_test_split
X,X_final,Y,Y_final=train_test_split(X,Y,test_size=0.1)
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25)
SC=StandardScaler()
X_train=SC.fit_transform(X_train)
V1_test,V2_test,Y1_test,Y2_test=train_test_split(X_test,Y_test,test_size=0.33)
v1_test=SC.fit_transform(V1_test)
V2_test=SC.fit_transform(V2_test)

In [14]:

V1_test.shape[0]

6815

In [15]:
from joblib import dump,load
from sklearn .linear_model import LogisticRegression
class Tree1:
  def __init__(self,loss,ind,rows):
    self.left=None
    self.right=None
    self.loss=loss
    self.ind=ind
    self.rows=rows
    self.par=None
    self.var=None
    self.threshold=None
  def fun(self,clf):
    dump(clf,"model"+str(self.ind))
  def load(self):
    return load("model"+str(self.ind))
clf=LogisticRegression().fit(X_train,Y_train)
#clf.score(V1_test,Y1_test)


X_train=pd.DataFrame(X_train)
from sklearn.metrics import log_loss
from queue import Queue
q=Queue(0)
fpr,tpr,threshold=metrics.roc_curve(Y1_test,clf.predict(V1_test))
loss=1-metrics.auc(fpr,tpr)
ind=1
rows=[]
for i in range (X_train.shape[0]):
  rows.append(i)

root=Tree1(loss,ind,rows)
root.fun(clf)
q.put(root)
terminals=[]

while not q.empty():
  print(q.qsize())
  temp=q.get()
  loss_min=temp.loss
  if(len(temp.rows))<100:
    terminals.append(temp)
    continue
  count=0
  for xi in X_train.columns:
    count=count+1
    print(count)
    row1=[]
    row2=[]
    min1=min(X_train.iloc[temp.rows,xi])
    max1=max(X_train.iloc[temp.rows,xi])
    threshold=min1+(max1-min1)/10
    for i in temp.rows:
      if X_train.iloc[i,xi]<threshold :
        row1.append(i)
      else:
        row2.append(i)
    if len(row2)<100 or len(row1)<100:
        continue
    X_train_left=X_train.iloc[row1,:]
    Y_train_left=Y_train.iloc[row1,:]
    X_train_right=X_train.iloc[row2,:]
    Y_train_right=Y_train.iloc[row2,:]
    a=0
    b=0
    for i in range (Y_train_left.shape[0]):
      if(Y_train_left.iloc[i,0]==0):
        a=a+1
      else:
        b=b+1
    if(a==0 or b==0):
      continue
    a=0
    b=0
    for i in range (Y_train_right.shape[0]):
      if(Y_train_right.iloc[i,0]==0):
        a=a+1
      else:
        b=b+1 
    if(a==0 or b==0):
      continue
    clf_left=LogisticRegression().fit(X_train_left, Y_train_left)
    clf_right=LogisticRegression().fit(X_train_right, Y_train_right)
    fpr1,tpr1,threshold1=metrics.roc_curve(Y1_test,clf_left.predict(V1_test))
    fpr2,tpr2,threshold2=metrics.roc_curve(Y1_test,clf_right.predict(V1_test))
    loss1=1-metrics.auc(fpr1,tpr1)
    loss2=1-metrics.auc(fpr2,tpr2)
    if((loss1+loss2)/2)<loss_min :
      temp.var=xi
      loss_min=(loss1+loss2)/2
      temp.threshold=threshold
  if temp.loss==loss_min:
    terminals.append(temp)
    continue
  xi=temp.var
  row1=[]
  row2=[]
  threshold=temp.threshold
  for i in temp.rows:
    if X_train.iloc[i,xi]<threshold :
        row1.append(i)
    else:
        row2.append(i)
  X_train_left=X_train.iloc[row1,:]
  Y_train_left=Y_train.iloc[row1,:]
  X_train_right=X_train.iloc[row2,:]
  Y_train_right=Y_train.iloc[row2,:]
  clf_left=LogisticRegression().fit(X_train_left, Y_train_left)
  clf_right=LogisticRegression().fit(X_train_right, Y_train_right)
  fpr1,tpr1,threshold1=metrics.roc_curve(Y1_test,clf_left.predict(V1_test))
  fpr2,tpr2,threshold2=metrics.roc_curve(Y1_test,clf_right.predict(V1_test)) 
  loss1=1-metrics.auc(fpr1,tpr1)
  loss2=1-metrics.auc(fpr2,tpr2)
  ind=ind+1
  temp.left=Tree1(loss1,ind,row1)
  ind=ind+1
  temp.right=Tree1(loss2,ind,row2)
  q.put(temp.left)
  q.put(temp.right)
  temp.left.fun(clf_left)
  temp.left.par=temp
  temp.right.fun(clf_right)
  temp.right.par=temp
  



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1
1


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

2


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
3
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


29
30
31
32
33
34
35
36
37
38
39
40
41
42


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


43
44
45
46
47
48
1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [16]:
q1=Queue(0)
q1.put(root)
a=0
for temp in terminals:
  print(len(temp.rows))
  print('\n')
print(a)

553


245


893


369


728


199


1482


276


507


100


265


133


100


244


216


454


231


212


155


249


1188


286


499


171


169


321


105


116


203


219


285


353


112


6516


105


279


128


141


111


299


292


1191


342


2969


113


254


328


125


186


374


462


203


181


980


150


2568


107


144


224


106


0


In [17]:
len(terminals)

60

In [18]:
dictionary={}
count=0
for terminal in terminals:
  dictionary[terminal.ind]=count
  count=count+1
predictor=[]
for terminal in terminals:
  par=[]
  temp_node=terminal
  while temp_node!=None:
    par.append(temp_node.load())
    temp_node=temp_node.par
  pred=[]
  par.reverse()
  for i in par:
    pred.append(i.predict(V2_test))
  pred=np.transpose(pred)
  pred=pd.DataFrame(pred)
  pred
  clf=LogisticRegression().fit(pred,Y2_test)
  predictor.append(clf)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [19]:
'''dataset1=pd.read_csv('bank.csv',sep=';')
dataset1=pd.DataFrame(dataset1)
Y_final=dataset1.iloc[:,-1]
X_final=dataset1.iloc[:,:-1]
X_final=X_final.drop(['contact'],axis='columns')
enc=OneHotEncoder()
X_final=pd.get_dummies(X_final)
X_final=pd.DataFrame(X_final)
Y_final=pd.get_dummies(Y_final);
Y_final=Y_final.drop(['no'],axis='columns')'''
Sc=StandardScaler()
X_final=Sc.fit_transform(X_final)
Y_finalpred=[]


In [20]:
X_final=pd.DataFrame(X_final)
X_final

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,1.483273,-0.363884,0.876693,0.120983,1.965673,-0.422755,-0.315040,-0.341384,-0.528401,-0.185228,-0.160817,1.917373,-0.228828,-0.195214,-0.316189,-0.144109,-0.456557,-0.16999,-0.083082,-0.361639,0.807947,-0.622529,-0.432292,-1.004433,1.518406,-0.198246,0.134201,-0.134201,1.100076,-1.100076,-2.354600,2.354600,-0.255938,-0.416325,-0.073046,-0.252982,-0.165812,2.438491,-0.368640,-0.104664,-0.666534,-0.308083,-0.131616,-0.113986,-0.351395,-0.213988,-0.198246,0.486694
1,1.204488,0.072333,-1.402529,-0.922709,-0.562869,-0.422755,-0.315040,-0.341384,1.892501,-0.185228,-0.160817,-0.521547,-0.228828,-0.195214,-0.316189,-0.144109,-0.456557,-0.16999,-0.083082,-0.361639,-1.237705,1.606351,-0.432292,0.995587,-0.658586,-0.198246,0.134201,-0.134201,-0.909028,0.909028,0.424701,-0.424701,-0.255938,-0.416325,-0.073046,-0.252982,-0.165812,-0.410090,-0.368640,-0.104664,-0.666534,-0.308083,7.597847,-0.113986,-0.351395,-0.213988,-0.198246,0.486694
2,0.368136,-0.397107,0.396857,-0.480275,2.597808,-0.422755,-0.315040,-0.341384,-0.528401,5.398765,-0.160817,-0.521547,-0.228828,-0.195214,-0.316189,-0.144109,-0.456557,-0.16999,-0.083082,-0.361639,0.807947,-0.622529,-0.432292,0.995587,-0.658586,-0.198246,0.134201,-0.134201,1.100076,-1.100076,0.424701,-0.424701,-0.255938,2.401970,-0.073046,-0.252982,-0.165812,-0.410090,-0.368640,-0.104664,-0.666534,-0.308083,-0.131616,-0.113986,-0.351395,-0.213988,-0.198246,0.486694
3,2.784265,8.780660,0.276898,-0.665568,-0.562869,0.276324,1.239648,-0.341384,-0.528401,-0.185228,-0.160817,-0.521547,4.370100,-0.195214,-0.316189,-0.144109,-0.456557,-0.16999,-0.083082,-0.361639,0.807947,-0.622529,-0.432292,0.995587,-0.658586,-0.198246,0.134201,-0.134201,1.100076,-1.100076,0.424701,-0.424701,-0.255938,-0.416325,-0.073046,-0.252982,-0.165812,-0.410090,2.712673,-0.104664,-0.666534,-0.308083,-0.131616,-0.113986,-0.351395,4.673156,-0.198246,-2.054680
4,-0.747000,-0.016264,1.596447,0.003756,-0.562869,2.130823,1.239648,-0.341384,-0.528401,-0.185228,-0.160817,1.917373,-0.228828,-0.195214,-0.316189,-0.144109,-0.456557,-0.16999,-0.083082,-0.361639,-1.237705,1.606351,-0.432292,-1.004433,1.518406,-0.198246,0.134201,-0.134201,-0.909028,0.909028,0.424701,-0.424701,-0.255938,-0.416325,-0.073046,-0.252982,6.030912,-0.410090,-0.368640,-0.104664,-0.666534,-0.308083,-0.131616,-0.113986,2.845802,-0.213988,-0.198246,-2.054680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4517,-0.468216,-0.308203,0.636775,-0.726072,-0.246801,0.557897,2.276107,-0.341384,-0.528401,-0.185228,-0.160817,1.917373,-0.228828,-0.195214,-0.316189,-0.144109,-0.456557,-0.16999,-0.083082,-0.361639,0.807947,-0.622529,-0.432292,-1.004433,1.518406,-0.198246,0.134201,-0.134201,1.100076,-1.100076,0.424701,-0.424701,-0.255938,-0.416325,-0.073046,-0.252982,-0.165812,-0.410090,-0.368640,-0.104664,-0.666534,3.245877,-0.131616,-0.113986,2.845802,-0.213988,-0.198246,-2.054680
4518,1.111560,0.743576,0.996652,-0.071873,0.069266,-0.422755,-0.315040,-0.341384,1.892501,-0.185228,-0.160817,-0.521547,-0.228828,-0.195214,-0.316189,-0.144109,-0.456557,-0.16999,-0.083082,-0.361639,0.807947,-0.622529,-0.432292,0.995587,-0.658586,-0.198246,0.134201,-0.134201,1.100076,-1.100076,0.424701,-0.424701,-0.255938,-0.416325,-0.073046,-0.252982,-0.165812,-0.410090,2.712673,-0.104664,-0.666534,-0.308083,-0.131616,-0.113986,-0.351395,-0.213988,-0.198246,0.486694
4519,0.275208,-0.406029,-1.042651,-0.506745,-0.562869,3.053219,0.203189,-0.341384,-0.528401,-0.185228,-0.160817,-0.521547,-0.228828,-0.195214,-0.316189,-0.144109,2.190305,-0.16999,-0.083082,-0.361639,0.807947,-0.622529,-0.432292,0.995587,-0.658586,-0.198246,0.134201,-0.134201,-0.909028,0.909028,0.424701,-0.424701,-0.255938,-0.416325,-0.073046,-0.252982,-0.165812,-0.410090,-0

In [21]:

def final_prediction(X):
  temp=root
  final=[]
  clf=temp.load()
  final.append((clf.predict(X)).reshape(-1,1))
  while(temp.left!=None):
    if(X.iloc[0,temp.var]<temp.threshold):
      temp=temp.left
    else:
      temp=temp.right
    clf=temp.load()
    final.append((clf.predict(X)).reshape(-1,1))
  final=np.array(final)
  final=final.reshape(1,final.shape[0])
  return predictor[dictionary[temp.ind]].predict(final)



In [22]:
for i in range(X_final.shape[0]):
  Y_finalpred.append(final_prediction(X_final.iloc[[i],:]))

from sklearn.metrics import accuracy_score
accuracy_score(Y_final,Y_finalpred)


0.9080053073861123

In [23]:
fpr,tpr,_=metrics.roc_curve(Y_final,Y_finalpred)
1-metrics.auc(fpr,tpr)

0.34609345923389945